## Методы сбора и обработки данных из сети Интернет

### Парсинг HTML XPath

Пиложение, которое собирает основные новости с сайтов mail.ru, lenta.ru.
Для парсинга использован xpath. Структура данных содержит:
* название источника,
* наименование новости,
* ссылку на новость,
* дата публикации

In [48]:
from pprint import pprint
from lxml import html
import requests
from pprint import pprint
import pandas as pd
from datetime import datetime

In [49]:
link_lenta = 'https://lenta.ru'
link_mail = 'https://mail.ru'
headers = {'User-agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
           AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/77.0.3865.90 Safari/537.36'}

In [50]:
def get_news_from_lenta_ru(link, headers=headers):
    req = requests.get(link, headers=headers).text
    root = html.fromstring(req)   
    
    # time
    time = root.xpath(
        '//time[@class="g-time"]/text()'
    )
    raw_news = root.xpath(
        '//time[@class="g-time"]/../text()'
    )
    hrefs = root.xpath(
        '//time[@class="g-time"]/../@href'
    )
    
    # news
    news = []
    for item in raw_news:
        r = item.replace('\xa0',' ')
        news.append(r)
    
    # href
    href = []
    for item in hrefs:
        href.append(link + item)
    
    #site
    site = [link] * len(news)
        
    return list(zip(time, news, href, site))


def get_news_from_mail_ru(link, headers=headers):
    req = requests.get(link, headers=headers).text
    root = html.fromstring(req)
    
    # News
    keys= [0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16] # List of №-news scrapped
    n = root.xpath(
        '//div[contains(@class, "news-item")]//a[last()]//text()'
    )
    news = ([n[i] for i in keys]) # Scip extended text block from main_news
    
    # href
    href = root.xpath(
        '//div[contains(@class, "news-item")]//a[last()]//@href'
    )
    
    # time
    t = datetime.now()
    now = str(t.hour) + ':' + str(t.minute)    
    time = [now] * len(news)
    
    # site
    site = [link] * len(news) 
    
    return list(zip(time, news, href, site))


def make_df(data):
    cols = [
        'Time',
        'News',
        'href',
        'Resource'
    ]
     
    return pd.DataFrame(data, columns=cols)

In [51]:
df_lenta = make_df(get_news_from_lenta_ru(link_lenta))
df_mail = make_df(get_news_from_mail_ru(link_mail))

In [52]:
df_lenta

,Time,News,href,Resource
0,15:29,Лукашенко рассказал о попросившем помощи Зелен...,https://lenta.ru/news/2019/09/26/lukashenka/,https://lenta.ru
1,15:44,Лукашенко раскрыл роль Януковича в ухудшении о...,https://lenta.ru/news/2019/09/26/yanik/,https://lenta.ru
2,15:38,Объяснена природа тысячелетних мегаструктур,https://lenta.ru/news/2019/09/26/megastructure/,https://lenta.ru
3,15:35,Уточнена информация о запрете на курение на ро...,https://lenta.ru/news/2019/09/26/thankyouforsm...,https://lenta.ru
4,15:33,Эштон Кутчер отреагировал на заявления Деми Му...,https://lenta.ru/news/2019/09/26/ashton/,https://lenta.ru
5,15:23,«Честный» чиновник прятал в подвале тонны золота,https://lenta.ru/news/2019/09/26/gold/,https://lenta.ru
6,15:21,Huawei объединился с Россией вокруг российской...,https://lenta.ru/news/2019/09/26/hua/,https://lenta.ru
7,15:19,В МВД рассказали о благодарностях фанатов «Спа...,https://lenta.ru/news/2019/09/26/fan_spartak/,https://lenta.ru
8,15:18,Фанаты увидели скверные условия жизни популярн...,https://lenta.ru/news/2019/09/26/blogger/,https://lenta.ru
9,15:11,Найдена причина московских пробок,https://lenta.ruhttps://moslenta.ru/news/naide...,https://lenta.ru


In [53]:
df_mail

,Time,News,href,Resource
0,15:51,Умер экс-президент Франции Жак Ширак,https://news.mail.ru/society/38866362/?frommail=1,https://mail.ru
1,15:51,В России отказались судить Шварценеггера и Дже...,https://news.mail.ru/society/38864436/?frommail=1,https://mail.ru
2,15:51,В подвале китайского чиновника нашли более 13 ...,https://news.mail.ru/incident/38866696/?fromma...,https://mail.ru
3,15:51,Горящий в Норвегии российский траулер опрокину...,https://news.mail.ru/incident/38863892/?fromma...,https://mail.ru
4,15:51,"Белка остановила девушку, чтобы та спасла бель...",https://news.mail.ru/society/38865961/?frommai...,https://mail.ru
5,15:51,Мегаполисы мертвых в Гонконге показали с возду...,https://news.mail.ru/society/38863867/?frommai...,https://mail.ru
6,15:51,Астронавт сняла подлетающую к МКС подругу на б...,https://news.mail.ru/society/38868111/?frommai...,https://mail.ru
7,15:51,Россия потратит миллиарды на жилье для чиновников,https://news.mail.ru/economics/38862466/?fromm...,https://mail.ru
8,15:51,Прокуратура назвала незаконными «звонки для уч...,https://news.mail.ru/society/38860886/?frommail=1,https://mail.ru
9,15:51,Синоптики пообещали возвращение тепла в центр ...,https://news.mail.ru/society/38865284/?frommail=1,https://mail.ru
